In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from matplotlib import cm
import os

In [3]:
df = pd.read_spss('data/CASPIANI.sav',convert_categoricals=True)  
df_iii = pd.read_spss('data/CASPIAN III.sav',convert_categoricals=True) 
df_iv = pd.read_spss('data/caspian4_.sav',convert_categoricals=True)
df_v = pd.read_spss('data/last-caspian-v.sav',convert_categoricals=True) 
def rename_features(df1, caspian_number):
    df2 = df1.copy()
    # Rename the features in the dataframe
    if caspian_number == 5:
        df2.rename(columns={'weight_1': 'weight', 'height_2': 'height', 'universi': 'university','ap_9':'schoolType'}, inplace=True)
    elif caspian_number == 4:
        df2.drop('sex', axis=1, inplace=True)
        print(df2.columns)
        df2.rename(columns={'sex2': 'sex', 'weight_1': 'weight', 'height_2': 'height', 'University': 'university','ap_9':'schoolType'}, inplace=True)
    elif caspian_number == 3:
        df2.rename(columns={'weighte': 'weight', 'heighte': 'height', 'province': 'university', 'area':'region','p9':'schoolType'}, inplace=True)
    elif caspian_number == 1:
        df2.columns = df.columns.str.lower()
        df2.rename(columns={'univer': 'university','district':'region', 'schoolty':'schoolType'}, inplace=True)
        

    return df2
df = rename_features(df,1)
df_iv=rename_features(df_iv,4)
df_iii = rename_features(df_iii,3)
df_v = rename_features(df_v,5)
df['sex'] = df['sex'].apply(lambda x: 'Girl' if x == 'Female' else 'Boy' if x == 'Male' else x)
df['region'] = df['region'].apply(lambda x: 'urban' if x == 'Urban' else 'rural' if x == 'Rural' else x)
df_iii['sex'] = df_iii['sex'].apply(lambda x: 'Boy' if x == 'female' else 'Girl' if x == 'male' else x)
df_iv['sex'] = df_iv['sex'].apply(lambda x: 'Girl' if x == 'girl' else 'Boy' if x == 'boy' else x)
df_iii['schoolType']=df_iii['schoolType'].apply(lambda x: 'Public School' if x=='dolati' else 'Private School' if x=='gheyre entefaei' else 'Unknown') 
df_iv['schoolType']=df_iv['schoolType'].apply(lambda x: 'Public School' if x=='dolati' else 'Private School' if x=='gheyre entef' else 'Unknown') 
df_v['schoolType']=df_v['schoolType'].apply(lambda x: 'Public School' if x=='dolati' else 'Private School' if x=='gheyre entefai' else 'Unknown') 
df_dict = {'caspian_I': df, 'caspian_III': df_iii, 'caspian_IV': df_iv, 'caspian_V': df_v}
def preprocess(dataframes_dict):
    processed_dfs = {}  # Dictionary to store processed DataFrames
    for name, df_org in dataframes_dict.items():
        df = df_org.copy()

        # Filter age
        df = df[(df["age"] >= 7) & (df["age"] <= 18)]
        # please change the type of heught_1 and weight_1 in caspian4 to numeric if you can't do it directly uncomment two line below
        df['height'] = pd.to_numeric(df['height'], errors='coerce')
        df['weight'] = pd.to_numeric(df['weight'], errors='coerce')
        df["bmi1"] = df["weight"] / ((df["height"] / 100) ** 2)
        
        # Remove null tuples
        records_with_nulls = df[
            df[["weight", "height", "sex", "age"]].isna().any(axis=1)
        ]
        df = df.dropna(subset=["height", "weight", "sex"])
        print(
            f"Number of records with NaN value in weight or height in {name}: {len(records_with_nulls)}"
        )

        # Store the processed DataFrame in the new dictionary
        processed_dfs[name] = df

    return processed_dfs


# Usage
processed_dfs = preprocess(df_dict)

Index(['id2', 'University', 'region', 'cluster', 'cross', 'birth_ye',
       'sample_c', 'a_1', 'a_2', 'a_3',
       ...
       'HeiMeter', 'bmi', 'CATBMI', 'CATBMIoutlieer', 'dicBMI',
       'socialsupport', 'PA1', 'PA2', 'KVDM', 'obes'],
      dtype='object', length=410)
Number of records with NaN value in weight or height in caspian_I: 11
Number of records with NaN value in weight or height in caspian_III: 0
Number of records with NaN value in weight or height in caspian_IV: 143
Number of records with NaN value in weight or height in caspian_V: 128


In [ ]:
def growth_chart_without_smooth(processed_dfs, feature, percentiles, sex):
    for name, preprocess_df in processed_dfs.items():
        def calculate_percentiles(group):
            return np.percentile(group[feature], percentiles)

        # Calculate percentiles
        percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
            lambda g: pd.Series(calculate_percentiles(g), index=[f'Percentile_{p}' for p in percentiles])
        )
        
        percentile_results.reset_index(inplace=True)
        
        if percentile_results.empty:
            print(f"No percentile results for {name} with feature {feature}")
            continue

        # Filter based on sex
        if sex == 'Boy':
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Boy']
            gender = "Boy"

        else:
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Girl']
            gender = "Girl"

        if percentile_results_gender.empty:
            print(f"No data for {gender} in {name} with feature {feature}")
            continue

        ages = percentile_results_gender['age']
        plt.figure(figsize=(12, 8))

        for i, percentile in enumerate(percentiles):
            percentile_data = percentile_results_gender[f'Percentile_{percentile}']
            print(f"{percentile}th Percentile\nLength: {len(percentile_data)}\nData: {percentile_data.values}")
            # Ensure there’s data to plot
            if percentile_data.empty:
                print(f"No data for {percentile}th percentile for {gender} in {name}")
                continue
            
            # Plot the raw percentile data without smoothing
            plt.plot(ages, percentile_data, label=f'{percentile}th Percentile', color=plt.cm.plasma(i / len(percentiles)))
        
        plt.xlabel('Age')
        plt.ylabel(feature)
        plt.title(f'Growth Chart for {gender} {name}')
        plt.legend()
        plt.grid(True)
        
        # Check that saving folder exists
        saving_folder = f'charts/{name}-raw'
        if not os.path.exists(saving_folder):
            os.makedirs(saving_folder)
        
        # Save figure
        plt.savefig(f'{saving_folder}/{gender}-{feature}-age_raw-{name}.png')
        # plt.show()
        plt.close()
        print(f"Chart saved for {gender} in {name} with feature {feature}")
        return percentile_results

        

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from statsmodels.nonparametric.smoothers_lowess import lowess

def modified_z_score(data):
    """Calculate the modified z-score for each data point."""
    median = np.median(data)
    mad = np.median(np.abs(data - median))  # Median Absolute Deviation
    return 0.6745 * (data - median) / mad

def growth_chart_without_smooth(processed_dfs, feature, percentiles, sex):
    for name, preprocess_df in processed_dfs.items():
        def calculate_percentiles(group):
            return np.percentile(group[feature], percentiles)

        # Calculate percentiles
        percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
            lambda g: pd.Series(calculate_percentiles(g), index=[f'Percentile_{p}' for p in percentiles])
        )

        percentile_results.reset_index(inplace=True)

        if percentile_results.empty:
            print(f"No percentile results for {name} with feature {feature}")
            continue

        # Filter based on sex
        if sex == 'Boy':
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Boy']
            gender = "Boy"
        else:
            percentile_results_gender = percentile_results[percentile_results['sex'] == 'Girl']
            gender = "Girl"

        if percentile_results_gender.empty:
            print(f"No data for {gender} in {name} with feature {feature}")
            continue

        ages = percentile_results_gender['age']
        plt.figure(figsize=(12, 8))

        for i, percentile in enumerate(percentiles):
            percentile_data = percentile_results_gender[f'Percentile_{percentile}']
            print(f"{percentile}th Percentile\nLength: {len(percentile_data)}\nData: {percentile_data.values}")

            # Ensure there’s data to plot
            if percentile_data.empty:
                print(f"No data for {percentile}th percentile for {gender} in {name}")
                continue
            
            # Calculate modified z-scores and filter out outliers
            z_scores = modified_z_score(percentile_data)
            filtered_data = percentile_data[np.abs(z_scores) < 3.5]  # Keep data points with z-score < 3.5
            
            # Corresponding ages for filtered data
            filtered_ages = ages[np.abs(z_scores) < 3.5]

            # Apply LOESS smoothing
            smoothed_data = lowess(filtered_data, filtered_ages, frac=0.3)  # Adjust frac for smoother curve
            smoothed_ages, smoothed_values = zip(*smoothed_data)

            # Plot the smoothed data
            plt.plot(smoothed_ages, smoothed_values, label=f'Smoothed {percentile}th Percentile', color=plt.cm.plasma(i / len(percentiles)))

        plt.xlabel('Age')
        plt.ylabel(feature)
        plt.title(f'Growth Chart for {gender} {name}')
        plt.legend()
        plt.grid(True)

        # Check that saving folder exists
        saving_folder = f'charts/chart_caspicans/{name}-smoothed'
        if not os.path.exists(saving_folder):
            os.makedirs(saving_folder)

        # Save figure
        plt.savefig(f'{saving_folder}/{gender}-{feature}-age_smoothed-{name}.png')
        plt.close()
        print(f"Chart saved for {gender} in {name} with feature {feature}")
        
        return percentile_results


In [5]:
percentiles=[3, 5, 10 , 25, 50, 75, 90, 95, 97]

In [9]:
growth_chart_without_smooth(processed_dfs,'height',percentiles,'Boy')

/tmp/ipykernel_73525/2277484587.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(
/tmp/ipykernel_73525/2277484587.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  percentile_results = preprocess_df.groupby(['age', 'sex'], group_keys=False).apply(


3th Percentile
Length: 12
Data: [112.74 115.95 122.   126.   130.   136.33 141.34 148.   148.72 156.
 155.   157.  ]
5th Percentile
Length: 12
Data: [113.   118.25 123.   127.   133.   138.   143.8  150.   152.   158.1
 156.   158.  ]
10th Percentile
Length: 12
Data: [116. 122. 126. 130. 135. 141. 147. 153. 157. 162. 160. 159.]
25th Percentile
Length: 12
Data: [120.  126.  131.  135.  140.  147.  154.5 158.  163.  168.  166.  163. ]
50th Percentile
Length: 12
Data: [126.  130.  135.  138.5 146.  152.  160.  165.  170.  173.  172.  171. ]
75th Percentile
Length: 12
Data: [129. 134. 140. 144. 151. 158. 168. 171. 176. 178. 177. 178.]
90th Percentile
Length: 12
Data: [132.2 139.5 143.  149.8 155.  163.9 173.  176.  181.  183.  181.2 180. ]
95th Percentile
Length: 12
Data: [135.1  142.75 146.5  153.   160.   169.9  176.   178.   183.8  184.
 183.   182.  ]
97th Percentile
Length: 12
Data: [138.26 145.   147.   153.   162.   172.67 177.   180.   185.28 185.97
 184.   182.8 ]
Chart saved for 

,age,sex,Percentile_3,Percentile_5,Percentile_10,Percentile_25,Percentile_50,Percentile_75,Percentile_90,Percentile_95,Percentile_97
0,7.0,Girl,115.40,116.00,117.0,121.00,124.0,128.0,132.0,134.00,138.00
1,7.0,Boy,112.74,113.00,116.0,120.00,126.0,129.0,132.2,135.10,138.26
2,8.0,Girl,116.00,119.00,121.0,125.00,129.0,134.0,137.0,140.00,141.60
3,8.0,Boy,115.95,118.25,122.0,126.00,130.0,134.0,139.5,142.75,145.00
4,9.0,Girl,123.00,124.00,125.0,130.75,134.0,140.0,145.0,147.05,150.63
5,9.0,Boy,122.00,123.00,126.0,131.00,135.0,140.0,143.0,146.50,147.00
6,10.0,Girl,129.00,130.00,132.0,137.00,142.0,146.5,153.0,156.30,158.58
7,10.0,Boy,126.00,127.00,130.0,135.00,138.5,144.0,149.8,153.00,153.00
8,11.0,Girl,132.00,135.80,139.6,144.00,149.0,155.0,159.0,160.00,162.00
9,11.0,Boy,130.00,133.00,135.0,140.00,146.0,151.0,155.0,160.00,162.00
